# 分子のシミュレーション

Kazuki Tsuoka, Kifumi Numata (Aug 18, 2023)

## Step 1: 水素分子の構造を定義する
このチュートリアルでは、水素分子(H$_2$)を使います。

<img src="h2.png"/>


まず最初に行うことは、各原子核の位置を固定します。pythonの原子核のリストとして指定し、各原子核（リスト）には、原子の種類に対応した文字列とその3次元座標（別のリスト）が含まれます。また、全体の電荷を指定すると、その電荷を生成するために必要な電子の数をQiskitが自動的に計算します。

原子核の位置が固定されると（原子核-原子核間の力は一時的に無関係になり）、Qiskit は相互作用する電子-電子間のエネルギーの式(ハミルトニアン)を生成します。

ここでは、電子のエネルギー式(ハミルトニアン)を計算するための古典的な化学のコードとのインターフェースとなる「ドライバー」を指定します。

In [ ]:
h0 = "H 0 0 0; "
h1 = "H 0 0 0.735; "
geometry = h0 + h1
print(geometry)

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
driver = PySCFDriver(atom=geometry, charge=0, basis="sto-3g")

## Step 2: 電子の運動のエネルギーの式を作る

電子のハミルトニアンを構築するためにドライバーを呼び出します。

In [ ]:
# PySCF を使用してセットアップ計算を実行し、計算からデータを取得します
problem = driver.run()

ここでは、電子(フェルミオン：フェルミオンとは、電子の振る舞いを表す言葉です)のハミルトニアン（フェルミ演算子によるハミルトニアン）の作成を進めるように指示しています。

In [ ]:
# 第二量子化ハミルトニアンを取得する
fermionic_op = problem.hamiltonian.second_q_op()
print(fermionic_op)

In [ ]:
# 電子の軌道と電子の数の数は以下のように計算されます。
num_spatial_orbitals=problem.num_spatial_orbitals
num_particles= problem.num_particles
print(num_spatial_orbitals, num_particles)

## Step 3: エネルギー式を量子ゲートに変換

量子コンピューターは量子ビットで作られているため、このフェルミ演算子のハミルトニアンを量子ビット演算子に変換する必要があります。フェルミ演算子から量子ビット演算子への変換器は複数あります。色々な変換器を試すことができますが、ここでは、量子ビットが電子のスピン軌道を表現するというシンプルな関係性を持つJordanWignerMapperを使います。

以下で、フェルミ演算子によるハミルトニアンをパウリ演算子に変換します。

In [ ]:
from qiskit_nature import settings
settings.use_pauli_sum_op = False

from qiskit_nature.second_q.mappers import JordanWignerMapper
mapper = JordanWignerMapper()
qubit_op = mapper.map(fermionic_op)
print(qubit_op)

さて、分子とその量子コンピューターへのマッピングを定義したところで、基底エネルギー問題を解くためのアルゴリズムとして、変分量子固有値ソルバー(VQE)を使います。VQEは、短い量子回路を用いた量子-古典ハイブリッドアルゴリズムであり、現在のノイズのある量子コンピューターに適しています。現在の量子コンピューターは、ノイズによって結果が完全にかき消されてしまう前にしか、つまり、短時間しか計算が実行できません。

<img src="vqe_method_NB.png"/>

VQEは量子と古典の2つの計算から構成されています。

まず、量子コンピューター上に分子のトライアルの量子状態を作ります。このトライアル状態は、古典計算で調整されるパラメーターを含んでいます。トライアル状態が作成されると、そのエネルギーが量子コンピューター上で計算され、その結果は最終的に古典的に利用可能な結果となります。

古典コンピューターでは、古典的な最適化アルゴリズムが、前の状態のエネルギーレベルと新しいエネルギーレベルを比べ、トライアル状態のパラメーターを調整します。このプロセスは、エネルギーが最小になり変化しなくなるまで繰り返されます。アルゴリズム全体の出力は、基底エネルギーの近似値とそのパラメーターのセットです。


## Step 4:   トライアル状態(Ansatz)をセットする

トライアル状態は、（ランダムに初期化された）パラメーターの集まりによって指定され、基底エネルギー状態を探索（最小化）しながら状態をパラメーターを変化させていきます。トライアル状態の作成手法はいくつかありますが、今回は、化学の標準的な手法(UCCSD, ユニタリー結合クラスター singles/doubles法)を使います。

In [ ]:
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
ansatz = UCCSD(
    num_spatial_orbitals,
    num_particles,
    mapper,
    initial_state=HartreeFock(
        num_spatial_orbitals,
        num_particles,
        mapper,
    ),
)
ansatz.draw("mpl")

In [ ]:
ansatz.decompose().decompose().decompose().draw("mpl")

## Step 5:  VQEの実行

Qiskit Natureは分子の基底状態を計算するためにGroundStateEigensolverというクラスを用意しています。

GroundStateEigensolver を使うために、VQEソルバーを設定します。

In [ ]:
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
# VQEソルバーの設定
vqe_solver = VQE(Estimator(), ansatz, SLSQP())

これでようやく、分子の基底状態のエネルギーを求める準備が整いました。

Step 2で作成した`problem`に対して、`GroundStateEigensolver` を使って基底状態（の近似）を得ます。

In [ ]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(problem)
print(res)

ご覧のように、電子分布の基底状態のエネルギーを計算しました：-1.85 Hartree

原子核の配置から、原子核エネルギー（Nuclear repulsion energy）も与えられています（古典計算から求められます）。

最後に、量子力学的に計算されたエネルギーと、それと同等の精度（ただし時間はかかる）で古典計算されたエネルギーを比較してみます。

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

In [ ]:
calc = GroundStateEigensolver(mapper, numpy_solver)
res = calc.solve(problem)
print(res)

Total ground state energyが、小数点以下において何桁も一致していることから、この特定のハミルトニアンに対して、VQEアルゴリズムが正確に最低のエネルギー解を見つけていることが分かります。

## Step 6：原子間距離を変えてエネルギー曲面を求める

Step 5では、与えられた分子に対して一度だけ ground_state solver を実行しましたが、上で紹介した仕様を使ってQiskitに分子の形状を変化させるように指示する方法はまだ説明していません。原子核の位置を変えて全エネルギーレベルを比較することで、最もエネルギーが低い原子核の配置を見つける方法です。最も低いエネルギーが「無限大」で**ない**場合、これはエネルギーが最小となる分子の「安定した」結合状態に相当します。このように、エネルギーを原子間距離の関数として求めることは、非常に重要な研究対象です。



In [ ]:
# 結合長を0.5から2.5まで0.1 Angstromずつ変化させます
h2_length = np.arange(0.5, 2.5, 0.1)

In [ ]:
# エネルギーのリスト
uccsd_energy_list=[]

In [ ]:
# 異なる結合長をループで計算
for bond_length in h2_length:
    new_string = "H 0 0 0; H 0 0 "+str(bond_length) 
    driver = PySCFDriver(
        atom=new_string,
        basis="sto3g",
    )
    problem = driver.run()
    initial_state = HartreeFock(
        problem.num_spatial_orbitals, 
        problem.num_particles, 
        mapper
    )
    ansatz = UCCSD(
        problem.num_spatial_orbitals, 
        problem.num_particles, 
        mapper, 
        initial_state=initial_state,
    )
    vqe_solver = VQE(Estimator(), ansatz, SLSQP())
    calc = GroundStateEigensolver(mapper, vqe_solver)
    res = calc.solve(problem)
    uccsd_energy_list += [res.total_energies[0]] 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(h2_length,uccsd_energy_list)
plt.title("Potential energy curve of hydrogen molecule")
plt.xlabel("H-H bond length")
plt.ylabel("Energy (hartrees)")

直感的には、エネルギー地形を、山と谷、そしてボールが転がる台地の隣にあると考えるとよいでしょう（ボールのx座標は、2つの水素原子の間隔に対応します）。ボールが台地を転がる速度が速すぎなければ（右から左へ）、ボールは谷に落ち着くかもしれません。傾きが正（2つの水素原子の間に引力があることを表す）なので、ボールはゆっくりと台地を転がり落ちます。ボールが谷である最小点を通り越すと、山の急な負の傾きの斜面にぶつかり、すぐに転がり戻ってしまいます（水素原子は互いに反発します）。

ここで、最小値がゼロであることに注目してください。これは、水素分子の原子核の位置を、既知の基底状態の位置に定義したからです。



***


# 量子化学でHIVの課題を解く

[IBM Quantum Challenge Africa](https://github.com/qiskit-community/ibm-quantum-challenge-africa-2021) より

<img src="HIV-1_capsid_wikipedia.png"/>

**HIVは、世界的に見ても公衆衛生上、非常に大きな課題となっているウイルスです。** その結果、栄養状態、健康へのアクセス、教育、研究費など、社会のさまざまな側面に影響を与えています。さらに、このウイルスは急速に変異し、異なる株が異なる地域に分布していることも問題です。特に、HIV-1-CとHIV-2の株は、主にアフリカで多く見られます。資金面での格差により、アフリカ株の治療法の研究は他のプログラムに比べて遅れています。アフリカの研究者たちは、この不均衡を解消しようと努力していますが、彼らのツールキットに量子コンピューティングなどの最新技術を加えることを検討すべきです。

**量子コンピューターは、医薬品の設計を飛躍的に向上させます。** 特に、新しい抗レトロウイルス剤を設計するためには、**化学シミュレーション** を行って、抗レトロウイルス剤がウイルスのタンパク質と結合することを確認することが重要です。このようなシミュレーションは、従来のスーパーコンピューターでは難しく、効果がないこともありました。しかし、量子コンピューターを使えば、より正確なシミュレーションが可能になり、よりよい薬剤設計のワークフローが実現すると期待されています。

詳細：抗レトロウイルス薬は、ウイルスのポリタンパク質をより小さなタンパク質に切断してパッケージ化するプロテアーゼと呼ばれるウイルスタンパク質に結合してブロックする薬剤です。プロテアーゼは、化学的なハサミと考えることができます。抗レトロウイルス剤は、ハサミの切断能力を阻害する粘着性の障害物と考えることができます。プロテアーゼが阻害されると、ウイルスは自分自身のコピーを増やすことができません。

ウイルスのプロテアーゼに変異があると、ある抗レトロウイルスの結合傾向が変わります。そのため、変異が起きて抗レトロウイルスがうまく結合できなくなった場合、抗レトロウイルスの分子を調整して再び強く結合できるようにすることが目標となります。

**この課題の主な目的は、抗レトロウイルス分子のトイモデルが、ウイルスのプロテアーゼのトイモデルと結合するかどうかを調べることです。**

この課題では、 **最先端のハイブリッド古典-量子組み込み化学モデリング** を導入し、古典的な近似とより正確な量子計算の間で作業負荷を分割することができます。

最後に、現在の量子コンピューティングに存在するノイズを考慮して最高のパフォーマンスを達成するために、（量子コンピューティングの基本を理解していなくても）量子化学アルゴリズムの設定を微調整する必要があります。

## プロテアーゼ＋抗レトロウイルス薬のマクロ分子の特定

### プロテアーゼ

実際のプロテアーゼ分子は、約100個のアミノ酸からなる2本のポリペプチド鎖で構成されており（2本の鎖は折りたたまれています）、隣り合うアミノ酸同士はいわゆる*ペプチド結合*で結ばれています。

<img src="peptide_bond_wikipedia.png" title="Amino Acid bonding"/>

私たちのプロテアーゼ分子のトイモデルは、このペプチド結合からヒントを得ることにしました。ペプチド結合は、多数のアミノ酸を結合しタンパク質を作る基本的な構造です。ペプチド結合は、一般的なタンパク質の折り畳みや、HIVプロテアーゼの切断能力など、タンパク質の化学的性質を決定する最も重要な要素の1つです。

計算を簡単にするために、分子のO=C-Nの部分に注目してみましょう。分子をできるだけ現実的なものにするために、十分な量の水素原子(H)を残し、また追加します（実際、HCONH$_2$（ホルムアミド）は安定した分子で、イオン性の溶媒でもあるので、イオン結合を「切る」ことができます。)

O=C-Nをプロテアーゼ分子のトイモデルにすること、非常に単純化されてはいますが、それでも生物学的に動機付けされています。
これがそのプロテアーゼのトイモデルです：

<img width=50% src="protease.png"/>

```
"O":  (1.1280, 0.2091, 0.0000)
"N": (-1.1878, 0.1791, 0.0000)
"C": (0.0598, -0.3882, 0.0000)
"H": (-1.3085, 1.1864, 0.0001)
"H": (-2.0305, -0.3861, -0.0001)
"H": (-0.0014, -1.4883, -0.0001)
```

この分子をハサミに見立てて、HIVウイルスのコピーを作る過程で、HIVのマスタータンパク質（Gag-Pol高タンパク質）を切ることができると想像してみてください：

<img width=30% src="carpet_scissors_wikipedia_cropped.png"/>

### 抗レトロウイルス

抗レトロウイルス剤とは、プロテアーゼと結合して、その**切断機構を阻害する**分子のことです。今回のチャレンジでは、1個の炭素原子(C)を抗レトロウイルス分子の代用とします。

<img width=10% src="arv.png"/>

### マクロ分子
2つの分子は私たちの頭の中では別々になっていますが、接近すると、外側の電子がすべての原子の周りに分子軌道を形成して、1つのマクロ分子になります。

水素の例題で説明したように、量子的な電子分布は原子の位置を固定して計算されるので、原子を個別に配置する必要があります。1つ目と2つ目の課題では、プロテアーゼの座標を固定し、抗レトロウイルスの位置だけを直線に沿って変化させます。

その際、任意の点を通り、窒素原子(N)に近づくアプローチの線を選びます。この「ブロック」と呼ばれるアプローチは、ハサミが切るのを邪魔しようとするものです。それが「刺さった」となれば成功で、HIVの複製活動を妨害することに成功したことになります。

## Step 1: マクロ分子の分子定義

抗レトロウイルス剤が「刃」の間にある窒素原子（N）に接近する様子を表現するために、分子定義と分子変化を構築します：
<img width=50% src="arv_approaches_protease.png"/>
 ```
 "C": (-0.1805, 1.3955, 0.0000)
 ```

## 演習
下のセルにあなたの分子を作ってください。分子の名前は `macromolecule` とします。

In [ ]:
# Step 1: 分子の構造を定義する
o = "O 1.1280 0.2091 0.0000; "
n = "" # この原子名と座標を埋めます
c0 = "" # この原子名と座標を埋めます
h0 = "" # この原子名と座標を埋めます
h1 = "" # この原子名と座標を埋めます
h2 = "" # この原子名と座標を埋めます
c1 = "" # この原子名と座標を埋めます
macromolecule = o + n + c0 + h0 + h1 + h2+ c1
print(macromolecule)

In [ ]:
driver = PySCFDriver(atom = macromolecule, charge=0, basis="sto-3g")

In [ ]:
# Step 2：電子の運動のエネルギーの式を作る
es_problem = driver.run()
#fermionic_op = es_problem.hamiltonian.second_q_op()

## 量子ワークロードの削減

Step 2では、量子ビット・ハミルトニアンを構築しました。上記のマクロ分子にStep 2以降を適用しようとすると、基底状態計算シミュレーションは失敗します。なぜかというと、電荷を0に指定しましたが、Qiskitは30(=2\*6(2個の炭素)+7(窒素)+8(酸素)+3\*1(3つの水素))個の電子で動作しなければならないことを知っているからです。これは、第二量子化を行うと、60個のスピン軌道に変換されるので、60量子ビットが必要となります。60量子ビットは、私たちが古典シミュレーションできる範囲を超えており、60量子ビット以上のIBM量子システムもありますが、私たちは使えません。したがって、このチャレンジの目的のために、量子ビットの数を減らす必要があります。幸いなことに、この量子ビット数の削減は化学的な観点からも理由があります。正確な化学的結果を得るためには、内殻電子に対して古典的なハートリーフォック近似で十分な場合があります。さらに幸いなことに、Qiskitはつい最近拡張され、特定の電子を量子計算で処理し、残りの電子を古典的に近似するようにユーザーがシームレスに指定できるようになりました。今後、さらに多くの量子ビットが使えるようになったとしても、近未来の量子コンピューターが通常では手の届かない非常に大きな分子に取り組む際に、この機能は非常に有用であることがわかるでしょう。

*そこで次に、Qiskitに指示して、ある数の電子に量子計算処理をさせる方法を紹介します：*

In [ ]:
# ActiveSpaceTransformerを使用して問題のスペースサイズを削減します
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
as_transformer =  ActiveSpaceTransformer(2,2)
as_problem = as_transformer.transform(es_problem)

In [ ]:
# 電子の軌道と電子の数の数は以下のように計算されます
num_spatial_orbitals = as_problem.num_spatial_orbitals
num_particles= as_problem.num_particles
print(num_spatial_orbitals, num_particles)

上記のように、Qiskitには**ActiveSpaceTransformer**というクラスがあり、2つの引数を受け取ります。1つ目は、量子計算処理を受けるべき電子の数です（一番外側の電子から順に選んでいきます）。2つ目は、それらの電子が自由に動き回れるようにするための軌道の数です。必要な量子ビット数を決めるのは、この2つ目の数です。（NとCの結合なのでそれぞれ1個ずつ合計2個の電子をアクティブな電子として扱い、それ以外の内殻電子は非アクティブな電子とします。）

まずは、このまま基底エネルギーを出してみましょう。

In [ ]:
# Step 4: トライアル状態(Ansatz)をセットする
initial_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
ansatz = UCCSD(num_spatial_orbitals, num_particles, mapper, initial_state=initial_state,)

# Step 5: VQEの実行
vqe_solver = VQE(Estimator(), ansatz, SLSQP())
calc = GroundStateEigensolver(mapper, vqe_solver)
res_UCCSD_active = calc.solve(as_problem)
print('res_UCCSD_active : ', res_UCCSD_active)

## 抗レトロウイルス剤はプロテアーゼと結合し、反応をブロックするでしょうか？

抗レトロウイルス剤のトイモデルとプロテアーゼのトイモデルを使って、反応をシミュレーションします。

In [ ]:
# 結合長を-0.5から5まで0.5 Angstromずつ変化させます
hiv_length = np.arange(-0.5, 5, 0.5)

# エネルギーのリスト
uccsd_energy_list=[]

In [ ]:
def get_carbon_coordinate(bond_length: float):
    """結合長から炭素の座標を計算する関数"""
    slope = (1.3955 - 0.1791) / (-0.1805 + 1.1878)
    x_cood = bond_length / np.sqrt(1 + slope ** 2)
    y_cood = slope * x_cood
    return x_cood - 0.1805, y_cood + 1.3955

# 異なる結合長をループで計算
for bond_length in hiv_length:
    # Step 1: 分子の構造を定義する
    c1 = "C "+str(get_carbon_coordinate(bond_length)[0]) +" "+str(get_carbon_coordinate(bond_length)[1]) +" 0" 
    new_macromolecule = o + n + c0 + h0 + h1 + h2+ c1
    #print(new_macromolecule)
    driver = PySCFDriver(
        atom=new_macromolecule,
        charge=0,
        spin=0,
        basis="sto3g",
    )
    # Step 2: 電子の運動のエネルギーの式を作る
    es_problem = driver.run()
    # Step 4: トライアル状態(Ansatz)をセットする
    initial_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    ansatz = UCCSD(num_spatial_orbitals, num_particles, mapper, initial_state=initial_state,)
    # Step 5: VQEの実行
    vqe_solver = VQE(Estimator(), ansatz, SLSQP())
    calc = GroundStateEigensolver(mapper, vqe_solver)
    as_problem = as_transformer.transform(es_problem)
    res = calc.solve(as_problem)
    uccsd_energy_list += [res.total_energies[0]] 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hiv_length,uccsd_energy_list,color='red')
plt.ylim(-203.9, -203)
plt.title("Potential energy curve")
plt.xlabel("Atomic distance bond length")
plt.ylabel("Energy (hartrees)")

抗レトロウイルス剤がプロテアーゼに接近したときのエネルギー・ランドスケープが得られましたか? 抗レトロウイルス剤はプロテアーゼと結合し、反応をブロックするでしょうか?

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright